In [1]:
A = {1, 2, 3, 4, 5}
B = {4, 5, 6, 7, 8}

A^B

{1, 2, 3, 6, 7, 8}

In [1]:
import sys
import math
import random
from itertools import count
from random import choice
import tensorflow as tf
import numpy as np

from genome import DefaultGenomes
from gene import DefaultNodeGene, DefaultConnectionGene
from spices import Species, DefaultSpeciesSet
from network import FeedForwardNetwork
from mutate_crossover import DefaultReproduction

elitism = 20 # それぞれの種である世代から次の世代にコピーされるエリート個体の数
spice_elitism = 3 # 停滞から保護される種の数を示します。例えば3と設定すると、種の適応度の最も高い3つの種が、たとえ改善が示されなかったとしても、停滞により削除されなくなります。
survival_threshold = 0.2 # それぞれの種で交叉に使われるエリート個体の割合
min_species_size = 2 # それぞれの種の最小個体数
species_fitness_func = max # 種の適応度を計算する関数:max, min, meanなど
max_stagnation = 30 # 種が停滞すると見なされる世代数

num_inputs = 1
num_outputs = 784

pop_size = 100
fitness_theashold = 0.95

def _create_dataset():
    mnist = tf.keras.datasets.mnist
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    x_train = x_train.reshape(x_train.shape[0], -1)
    x_test = x_test.reshape(x_test.shape[0], -1)
    train_indices = np.where((y_train == 0) | (y_train == 1))[0]
    test_indices = np.where((y_test == 0) | (y_test == 1))[0]
    x_train = x_train[train_indices]
    y_train = y_train[train_indices]
    x_test = x_test[test_indices]
    y_test = y_test[test_indices]
    # データ数が多いので削減
    x_train = x_train[:1000]
    y_train = y_train[:1000]
    return (x_train, y_train), (x_test, y_test)

if __name__ == '__main__':
    population = DefaultGenomes.create_new(DefaultGenomes, pop_size)
    species_set = DefaultSpeciesSet()
    generation = 0
    (x, y), (x_test, y_test) = _create_dataset()

    while True:
        ### 1.種に分ける
        species_set.speciate(population, generation)

        if len(population.items()) == 0:
            print(1)
            print('Population is empty')
            break

        if len(species_set.species) == 0:
            print(1)
            print('Species is empty')
            break

        ### 2.適応度の評価
        loss = tf.keras.losses.BinaryCrossentropy(from_logits=False, axis=-1, reduction="none")
        acc = tf.keras.metrics.BinaryAccuracy()
        FeedForwardNetwork.eval_genomes(population, x, y, loss, acc)

        if len(population.items()) == 0:
            print(2)
            print('Population is empty')
            break

        if len(species_set.species) == 0:
            print(2)
            print('Species is empty')
            break

        best_genome_id, best_genome = max(population.items(), key=lambda x: x[1].fitness)

        if len(population.items()) == 0:
            print(3)
            print('Population is empty')
            break

        if len(species_set.species) == 0:
            print(3)
            print('Species is empty')
            break

        print(f'Generation: {generation}, Best Genome: {best_genome_id}, Fitness: {best_genome.fitness}, Acc: {best_genome.acc}')

        if best_genome.acc >= fitness_theashold:
            print('Threshold reached')
            break

        ### 3.交叉 & 突然変異
        reproduction = DefaultReproduction()
        population = reproduction.reproduce(species_set, pop_size, generation)

        if len(species_set.species) == 0:
            print(4)
            print('Species is empty')
            break

        if len(population.items()) == 0:
            print(4)
            print('Population is empty')
            break

        generation += 1

    # winner = best_genome
    # winner_id = best_genome_id

    # print(f'Winner: {winner_id}\n{winner}')
    # print(f'Winner Fitness: {winner.fitness}')
    # print(f'Winner Acc: {winner.acc}')
    

Generation: 0, Best Genome: 40, Fitness: -0.34998419880867004, Acc: 0.5074999928474426
Generation: 1, Best Genome: 46, Fitness: -0.22300691902637482, Acc: 0.5347825884819031
Generation: 2, Best Genome: 73, Fitness: -0.3172731399536133, Acc: 0.4885844886302948
Generation: 3, Best Genome: 4, Fitness: -0.35491517186164856, Acc: 0.5333333611488342
Generation: 4, Best Genome: 72, Fitness: -0.4095627963542938, Acc: 0.5333333611488342
Generation: 5, Best Genome: 36, Fitness: -0.10480702668428421, Acc: 0.5414414405822754
Generation: 6, Best Genome: 24, Fitness: -0.12795808911323547, Acc: 0.4743589758872986
Generation: 7, Best Genome: 41, Fitness: -0.2322022020816803, Acc: 0.5158730149269104
Generation: 8, Best Genome: 49, Fitness: -0.20710232853889465, Acc: 0.4806666672229767
Generation: 9, Best Genome: 81, Fitness: -0.17319069802761078, Acc: 0.4963414669036865
Generation: 10, Best Genome: 43, Fitness: -0.22208280861377716, Acc: 0.489393949508667
Generation: 11, Best Genome: 15, Fitness: -0.18